In [1]:
import pandas as pd

#### S3에서 데이터 불러오기

In [2]:
code = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/codeBook_v3.xlsx"
codeBook = pd.read_excel(code)

In [3]:
kp20 = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/KP2020.csv"
kp2020 = pd.read_csv(kp20, encoding='cp949', index_col=False)

In [4]:
kp21 = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/KP2021.csv"
kp2021 = pd.read_csv(kp21, encoding='cp949', index_col=False)

In [5]:
npa = "https://ais7.s3.ap-northeast-2.amazonaws.com/%EC%82%AC%EB%B3%B8(29)+-+%ED%99%9C%EC%9A%A9%EB%8D%B0%EC%9D%B4%ED%84%B0/NPA2020.csv"
npa2020 = pd.read_csv(npa, encoding='cp949', index_col=False)

#### concat

In [6]:
display(codeBook.loc[(codeBook.코드명 == "보이스피싱")])

,No,컬럼명,컬럼 그룹,코드명,코드값
57,58,EVT_CL_CD,사건종별코드,보이스피싱,215


In [7]:
# 보이스피싱 코드값에 해당하는 값만 추출
voice20 = kp2020.loc[(kp2020.EVT_CL_CD == 215)]
voice21 = kp2021.loc[(kp2021.EVT_CL_CD == 215)]
voice_npa20 = npa2020.loc[(npa2020.EVT_CL_CD == 215)]

In [8]:
print(f"voice20: {voice20.shape}, voice21: {voice21.shape}, voice_npa20: {voice_npa20.shape}")

voice20: (1660, 10), voice21: (33829, 10), voice_npa20: (10779, 10)


In [9]:
# kp2020, kp2021의 보이스피싱 컬럼만 먼저 concat
concat_voice = pd.concat([voice20, voice21])
# concat_voice.sample(10)

In [10]:
# RECV_CPLT_DM 20xx년 으로 변환을 위해 문자열앞에 20을 붙여줌
concat_voice["RECV_CPLT_DM"] = ["20" + concat_voice["RECV_CPLT_DM"].iloc[x] for x in range(len(concat_voice["RECV_CPLT_DM"]))]

# RECV_CPLT_DM 을 datetime 형변환
concat_voice["RECV_CPLT_DM"] = pd.to_datetime(concat_voice["RECV_CPLT_DM"])

# RECV_DEPT_NM 컬럼이 NPA_CL 컬럼값과 같아서 제거
concat_voice = concat_voice.drop(["RECV_DEPT_NM"], axis=1)

concat_voice.sample(5)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
1140423,2021-11-19 09:30:55,13,10,215,1.0,대전광역시 서구 탄방동 (행정:탄방동) 844,127.392650,36.346283,NaN
580748,2021-06-27 16:38:44,19,10,215,1.0,충청남도 공주시 반죽동 (행정:중학동) 346-2,127.119476,36.451436,NaN
2215308,2022-10-04 12:34:40,13,10,215,1.0,대전광역시 서구 복수동 (행정:복수동 ) 292-14,127.372972,36.308757,NaN
2015180,2022-08-16 14:26:22,13,10,215,3.0,NaN,127.404663,36.341685,NaN
11531,2020-12-07 10:45:47,19,10,215,2.0,충청남도 태안군 태안읍 평천리(행정:태안읍) 59-1,126.310780,36.750320,NaN


In [11]:
# RECV_CPLT_DT, RECV_CPLT_TM 를 datetime 형변환을 위해 str로 형변환
# concat_voice와 콘캣시 타입에 맞추기위해 float로 형변환
voice_npa20 = voice_npa20.astype({"RECV_CPLT_DT" : "str", "RECV_CPLT_TM" : "str", "RPTER_SEX" : "float"})
voice_npa20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10779 entries, 3221 to 1177944
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   RECV_CPLT_DT   10779 non-null  object 
 1   RECV_CPLT_TM   10779 non-null  object 
 2   NPA_CL         10779 non-null  int64  
 3   EVT_STAT_CD    10779 non-null  int64  
 4   EVT_CL_CD      10779 non-null  int64  
 5   RPTER_SEX      10779 non-null  float64
 6   HPPN_OLD_ADDR  10147 non-null  object 
 7   HPPN_X         10664 non-null  float64
 8   HPPN_Y         10664 non-null  float64
 9   SME_EVT_YN     6221 non-null   object 
dtypes: float64(3), int64(3), object(4)
memory usage: 926.3+ KB


In [12]:
# RECV_CPLT_TM 에 숫자가 6개가 아닐경우 0으로 채워줌
voice_npa20["RECV_CPLT_TM"] = [voice_npa20["RECV_CPLT_TM"].iloc[x].zfill(6) for x in range(len(voice_npa20["RECV_CPLT_TM"]))]

# RECV_CPLT_TM 시,분,초 단위로 나누기위해 슬라이싱을 통해 ':' 를 붙여줌
voice_npa20["RECV_CPLT_TM"] = [voice_npa20["RECV_CPLT_TM"].iloc[x][:2] + ":" + voice_npa20["RECV_CPLT_TM"].iloc[x][2:4] + ":" + voice_npa20["RECV_CPLT_TM"].iloc[x][4:] for x in range(len(voice_npa20["RECV_CPLT_TM"]))]

# 콘캣시키기 위해 RECV_CPLT_DM 컬럼을 생성하면서 RECV_CPLT_DT와 RECV_CPLT_TM를 합쳐줌
voice_npa20["RECV_CPLT_DM"] = voice_npa20["RECV_CPLT_DT"] + " " + voice_npa20["RECV_CPLT_TM"]

# RECV_CPLT_DM 을 datetime 형변환
voice_npa20["RECV_CPLT_DM"] = pd.to_datetime(voice_npa20["RECV_CPLT_DM"])

voice_npa20.sample(2)

,RECV_CPLT_DT,RECV_CPLT_TM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_OLD_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN,RECV_CPLT_DM
1089869,20201029,14:25:13,19,10,215,1.0,세종특별자치시 세종시 대평동(행정:대평동),127.277272,36.472146,,2020-10-29 14:25:13
923085,20200914,12:15:12,19,10,215,2.0,충청남도 천안시 동남구 신부동(행정:신안동) 85,127.157502,36.815245,,2020-09-14 12:15:12


In [13]:
# RECV_CPLT_DM을 생성하였으므로 RECV_CPLT_DT, RECV_CPLT_TM 는 drop
voice_npa20 = voice_npa20.drop(["RECV_CPLT_DT", "RECV_CPLT_TM"], axis=1)

# 콘캣시 컬럼이름값이 같아야 해서 컬럼이름 변경
voice_npa20 = voice_npa20.rename(columns={'HPPN_OLD_ADDR' : 'HPPN_PNU_ADDR'})

# concat_voice의 컬럼순서와 맞춤
voice_npa20 = voice_npa20[['RECV_CPLT_DM', 'NPA_CL', 'EVT_STAT_CD', 'EVT_CL_CD',
       'RPTER_SEX', 'HPPN_PNU_ADDR', 'HPPN_X', 'HPPN_Y', 'SME_EVT_YN']]

voice_npa20.sample(2)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
108431,2020-02-04 18:28:20,19,10,215,2.0,충청남도 천안시 서북구 쌍용동(행정:쌍용2동) 1552,127.114006,36.795942,NaN
432376,2020-03-17 00:20:07,19,10,215,2.0,NaN,0.000000,0.000000,NaN


In [14]:
# 머지이이
voice = pd.concat([concat_voice, voice_npa20])
# voice = merge_voice.merge(voice_npa20, how="left")
voice.sample(10)

,RECV_CPLT_DM,NPA_CL,EVT_STAT_CD,EVT_CL_CD,RPTER_SEX,HPPN_PNU_ADDR,HPPN_X,HPPN_Y,SME_EVT_YN
2173249,2022-09-19 16:15:27,13,10,215,3.0,대전광역시 서구 둔산동 (행정:둔산3동 ) 1876,127.395010,36.349682,NaN
1486638,2022-03-25 12:25:59,31,10,215,1.0,세종특별자치시 세종시 아름동(도담동) 103,127.247047,36.511463,Y
1198335,2021-12-13 17:09:08,13,10,215,3.0,NaN,127.404663,36.341685,Y
434503,2021-05-17 12:58:13,19,10,215,1.0,충청남도 천안시 서북구 두정동 (행정:부성1동) 401-1,127.156641,36.837076,NaN
2289322,2022-10-21 12:54:12,19,10,215,1.0,NaN,NaN,NaN,NaN
1168090,2021-12-02 19:24:26,19,10,215,2.0,충청남도 공주시 반포면 하신리(행정:반포면) 253-3,127.238214,36.382576,NaN
948559,2020-09-21 16:31:16,19,10,215,1.0,충청남도 천안시 서북구 입장면 시장리(행정:입장면) 4-11,127.216626,36.898712,Y
1313871,2022-01-26 12:14:15,13,10,215,1.0,대전광역시 유성구 송강동 (구즉동 ) 49-6,127.387226,36.434816,NaN
2398398,2022-11-16 16:17:44,19,10,215,1.0,NaN,NaN,NaN,NaN
270329,2021-04-02 12:08:28,19,10,215,2.0,충청남도 천안시 서북구 쌍용동(행정:쌍용2동) 494-17,127.124644,36.793748,NaN


#### NPA_CL -> 코드명

In [15]:
# 코드명으로 변경을 위해 str로 형변환
voice = voice.astype({"NPA_CL" : "str", "EVT_STAT_CD" : "str", "RPTER_SEX" : "str"})

In [16]:
voice.NPA_CL.unique()

array(['19', '13', '8', '31', '11', '15', '20', '23', '18', '12', '21',
       '9', '17', '10', '14', '16', '22'], dtype=object)

In [17]:
npa_cl_kor = {"19":"충남청", "13":"대전청", "31":"세종시", "11":"인천청", "20":"전북청", "23":"경남청", "18":"충북청", 
              "12":"광주청", "21":"전남청", "17":"강원청", "10":"대구청", "14":"울산청", "22":"경북청",  "15":"경기남부청", 
              "16":"경기북부청", "8":"서울청", "9":"부산청"}

voice["NPA_CL"] = voice["NPA_CL"].map(npa_cl_kor)

In [22]:
# NPA_CL 변경
voice["NPA_CL"] = voice["NPA_CL"].str.replace("19", "충남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("13", "대전청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("31", "세종시")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("11", "인천청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("15", "경기남부청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("20", "전북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("23", "경남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("18", "충북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("12", "광주청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("21", "전남청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("17", "강원청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("10", "대구청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("14", "울산청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("16", "경기북부청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("22", "경북청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("8", "서울청")
voice["NPA_CL"] = voice["NPA_CL"].str.replace("9", "부산청")

In [23]:
voice.NPA_CL.unique()

array(['충남청', '대전청', '서울청', '세종시', '인천청', '경기남부청', '전북청', '경남청', '충북청',
       '광주청', '전남청', '부산청', '강원청', '대구청', '울산청', '경기북부청', '경북청'],
      dtype=object)

#### EVT_STAT_CD -> 코드명

In [24]:
voice.EVT_STAT_CD.unique()

array(['10', '5', '7', '9'], dtype=object)

In [25]:
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("10", "종결요청")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("5", "도착")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("7", "종결요청")
voice["EVT_STAT_CD"] = voice["EVT_STAT_CD"].str.replace("9", "다청이첩")

#### RPTER_SEX -> 코드명

In [26]:
voice.RPTER_SEX.unique()

array(['1.0', '2.0', '3.0'], dtype=object)

In [27]:
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("1.0", "남성")
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("2.0", "여성")
voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("3.0", "불상")

C:\Users\tpfl1\AppData\Local\Temp\ipykernel_20460\3265531194.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("1.0", "남성")
C:\Users\tpfl1\AppData\Local\Temp\ipykernel_20460\3265531194.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("2.0", "여성")
C:\Users\tpfl1\AppData\Local\Temp\ipykernel_20460\3265531194.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  voice["RPTER_SEX"] = voice["RPTER_SEX"].str.replace("3.0", "불상")


#### save

In [28]:
path = r"C:\Users\tpfl1\OneDrive\Desktop\SERi"
voice.to_csv(f"{path}/voice.csv", index=False, encoding='cp949')